<a href="https://colab.research.google.com/github/SHIVRAJSHINDE/Kaggle/blob/main/SentimentAnalysisDistilBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn
!pip install googletrans==4.0.0-rc1


In [14]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4
!pip install textblob
!pip install mlxtend
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-9m8p4gom
  Running command git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-9m8p4gom
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 0f56a6502679f6750b7f0ffed98f590d1f0246bd
  Preparing metadata (setup.py) ... done
  Created wheel for preprocess_kgptalkie: filename=preprocess_kgptalkie-0.11-py3-none-any.whl size=8175 sha256=e83a7f900cf3157f9f4184fd6cfa859dcb70079918efe6e66e03d462c9c5fc02
  Stored in dir

In [15]:

# sentiment analysis with the pipeline
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")

data = ['i love you', 'i hate you']
sentiment_pipeline(data)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [1]:
# Data loading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import preprocess_kgptalkie as ps


In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/IMDB-Dataset.csv")
df = df.sample(10_000)
df.head()


,review,sentiment
24644,"While the ""date doctor"" concept is the one thi...",positive
28815,"Riff Randell is a wildly, obsessed fan of the ...",positive
34752,I saw 2:37 at the Toronto International Film F...,positive
24345,I think it was a pretty good film. It shows ho...,positive
4335,"I am an avid fan of Lucio Fulci, and yet I mus...",negative


In [4]:
df.shape
df.isnull().sum()


,0
review,0
sentiment,0


In [11]:
pip install nltk


In [12]:
# preprocessing
import preprocess_kgptalkie as ps
def get_word_counts(text):
    # Split the text into words using whitespace as the delimiter
    words = text.split()
    # Return the number of words
    return len(words)

def get_charcounts(text):
    # Return the number of characters in the text
    return len(text)

def get_avg_word_length(text):
    words = text.split()  # Split the text into words
    if not words:  # Handle the case where the text is empty or only contains whitespace
        return 0
    total_length = sum(len(word) for word in words)  # Calculate the total length of all words
    return total_length / len(words)  # Calculate and return the average length

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download the stopwords dataset if you haven't already
nltk.download('stopwords')
nltk.download('punkt')

def get_stopwords_counts(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())  # Convert to lowercase to match stopwords list
    stopword_count = sum(1 for word in words if word in stop_words)

    return stopword_count






[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
df['word_counts'] = df['review'].apply(lambda x: get_word_counts(x))
df['char_counts'] = df['review'].apply(lambda x: get_charcounts(x))
df['avg_wordlength'] = df['review'].apply(lambda x: get_avg_word_length(x))
df['stopwords_counts'] = df['review'].apply(lambda x: get_stopwords_counts(x))


df


,review,sentiment,word_counts,char_counts,avg_wordlength,stopwords_counts
24644,"While the ""date doctor"" concept is the one thi...",positive,399,2182,4.471178,199
28815,"Riff Randell is a wildly, obsessed fan of the ...",positive,765,4385,4.730719,348
34752,I saw 2:37 at the Toronto International Film F...,positive,165,1017,5.169697,73
24345,I think it was a pretty good film. It shows ho...,positive,131,716,4.473282,64
4335,"I am an avid fan of Lucio Fulci, and yet I mus...",negative,379,2151,4.678100,183
...,...,...,...,...,...,...
37749,I have witnessed some atrocities of cinema. In...,negative,307,1673,4.452769,162
48488,"Well, I can safely say I'm human, Wong. And I ...",positive,102,551,4.411765,48
31639,This is a very well done film showing the life...,positive,157,930,4.929936,76
44081,Feeling Minnesota is one of the worst films I ...,negative,174,950,4.465517,84


In [14]:

df.head()

df['review'] = df['review'].str.lower()
df['review'] = df['review'].apply(lambda x: ps.remove_html_tags(x))



/usr/local/lib/python3.10/dist-packages/preprocess_kgptalkie/text_preprocess.py:92: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(x, 'lxml').get_text()


In [15]:
df


,review,sentiment,word_counts,char_counts,avg_wordlength,stopwords_counts
24644,"while the ""date doctor"" concept is the one thi...",positive,399,2182,4.471178,199
28815,"riff randell is a wildly, obsessed fan of the ...",positive,765,4385,4.730719,348
34752,i saw 2:37 at the toronto international film f...,positive,165,1017,5.169697,73
24345,i think it was a pretty good film. it shows ho...,positive,131,716,4.473282,64
4335,"i am an avid fan of lucio fulci, and yet i mus...",negative,379,2151,4.678100,183
...,...,...,...,...,...,...
37749,i have witnessed some atrocities of cinema. in...,negative,307,1673,4.452769,162
48488,"well, i can safely say i'm human, wong. and i ...",positive,102,551,4.411765,48
31639,this is a very well done film showing the life...,positive,157,930,4.929936,76
44081,feeling minnesota is one of the worst films i ...,negative,174,950,4.465517,84
